# AEB Test Template
This python notebook provides an example to evaluate the AEB systems performance according to the following [reference](#https://cdn.euroncap.com/media/67254/euro-ncap-assessment-protocol-sa-v91.pdf).

The `TrialControl` class has been used to perform every Car-to-Car rear scenario.

In [1]:
from beamngpy import BeamNGpy
from beamng_ncap.scenarios import CCRS, CCRM, CCRB, generate_scenario

In [2]:
beamng = BeamNGpy('localhost', 64256)
beamng.open()
scenario = generate_scenario('etk800', 'etk800')
scenario.make(beamng)

beamng.set_deterministic()
beamng.set_steps_per_second(100)
beamng.load_scenario(scenario)
beamng.pause()
beamng.start_scenario()

### Run the CCRs tests

In [3]:
ccrs_scores = []
ccrs_states = []
overlap = 100
for vut_speed in range(10, 55, 5):
    print(f'Running CCRs test with VUT speed: {vut_speed} km/h')
    test = CCRS(beamng, vut_speed, overlap=overlap)
    sensors = test.load()
    test_state, test_score = test.execute('trial')
    ccrs_scores.append(test_score)
    ccrs_states.append(test_state)
    test.reset()

Running CCRs test with VUT speed: 10 km/h
Running CCRs test with VUT speed: 15 km/h
Running CCRs test with VUT speed: 20 km/h
Running CCRs test with VUT speed: 25 km/h
Running CCRs test with VUT speed: 30 km/h
Running CCRs test with VUT speed: 35 km/h
Running CCRs test with VUT speed: 40 km/h
Running CCRs test with VUT speed: 45 km/h
Running CCRs test with VUT speed: 50 km/h


### Run the CCRm tests

In [4]:
ccrm_scores = []
ccrm_states = []
overlap = 100
for vut_speed in range(30, 85, 5):
    print(f'Running CCRm test with VUT speed: {vut_speed} km/h')
    test = CCRM(beamng, vut_speed, overlap)
    sensors = test.load()
    test_state, test_score = test.execute('trial')
    ccrm_scores.append(test_score)
    ccrm_states.append(test_state)
    test.reset()

Running CCRm test with VUT speed: 30 km/h
Running CCRm test with VUT speed: 35 km/h
Running CCRm test with VUT speed: 40 km/h
Running CCRm test with VUT speed: 45 km/h
Running CCRm test with VUT speed: 50 km/h
Running CCRm test with VUT speed: 55 km/h
Running CCRm test with VUT speed: 60 km/h
Running CCRm test with VUT speed: 65 km/h
Running CCRm test with VUT speed: 70 km/h
Running CCRm test with VUT speed: 75 km/h
Running CCRm test with VUT speed: 80 km/h


### Run the CCRb tests

In [5]:
ccrb_scores = []
ccrb_states = []
overlap = 100
decelerations = [-2, -6]
distances = [12, 40]

for deceleration in decelerations:
    for distance in distances:
        print(f'Running CCRb test with distance: {distance} m and decelerations: {deceleration} m/s^2')
        test = CCRB(beamng, deceleration, distance, overlap)
        sensors = test.load()
        test_state, test_score = test.execute('trial')
        ccrb_scores.append(test_score)
        ccrb_states.append(test_state)
        test.reset()
        
beamng.close()

Running CCRb test with distance: 12 m and decelerations: -2 m/s^2
Running CCRb test with distance: 40 m and decelerations: -2 m/s^2
Running CCRb test with distance: 12 m and decelerations: -6 m/s^2
Running CCRb test with distance: 40 m and decelerations: -6 m/s^2


### Display the raw results

In [6]:
print(f'CCRs scores: {ccrs_scores}')
print(f'CCRm scores: {ccrm_scores}')
print(f'CCRb scores: {ccrb_scores}')

CCRs scores: [1, 1, 1, 1, 1, 1, 1, 1, 1]
CCRm scores: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
CCRb scores: [1, 1, 1, 1]


In [8]:
import numpy as np

ccrs_weights = np.array([1, 2, 2, 2, 2, 2, 1, 1, 1])
ccrm_weights = np.array([1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2])

ccrs_weighted_scores = ccrs_weights*np.array(ccrs_scores)
ccrm_weighted_scores = ccrm_weights*np.array(ccrm_scores)
ccrb_weighted_scores = np.array(ccrb_scores)

ccrs_total_score = np.sum(ccrs_weighted_scores)
ccrm_total_score = np.sum(ccrm_weighted_scores)
ccrb_total_score = np.sum(ccrb_weighted_scores)

ccrs_percentage = ccrs_total_score/14
ccrm_percentage = ccrm_total_score/15
ccrb_percentage = ccrb_total_score/4

AEB_score = (ccrs_percentage + ccrm_percentage + ccrb_percentage)/3*2

print(f'AEB score: {AEB_score} over 2')

AEB score: 2.0 over 2
